# `PatientAgent` demo notebook

In this notebook we introduce the patient agent and its methods including:
- initializing with comorbidities
- adding properties to conditions, such as severity
- updating the patient record
- the patient record internal reprsentation and converting to FHIR

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from pathlib import Path

PATIENT_ABM_DIR = Path(os.getcwd()).parent
PATIENT_ABM_DIR

In [ ]:
%env PATIENT_ABM_DIR={str(PATIENT_ABM_DIR)}

In [ ]:
# !pyreverse $PATIENT_ABM_DIR/src/patient_abm/agent/patient.py
# !dot -Tpng classes.dot -o classes.png

In [ ]:
import copy
import datetime
import json
import pprint
from dataclasses import asdict

from IPython.display import Image

from patient_abm.agent.patient import (
    PatientAgent, PatientRecordEntry, wrap_fhir_resource
)
from patient_abm.data_handler.fhir import (
    convert_patient_record_entry_to_fhir,
    generate_patient_fhir_resources,
    generate_patient_fhir_bundle,
    HAPI_FHIR_SERVER_4,
)
from patient_abm.utils import string_to_datetime

# Patient agent UML

Image("classes.png")

# Initialise patient agent

Below is the minimal input required to initialise a patient. `patient_id`, `gender` and `birth_date` are mandatory parameters.

In [ ]:
patient_id = "my-patient-0" # can be an integer or string, no underscores (for FHIR compliancy)
gender = "female" # must be either 'male' or 'female'
birth_date = "1985-05-24" # can be datetime string or datetime type 

In [ ]:
patient = PatientAgent(
    patient_id=patient_id,
    gender=gender,
    birth_date=birth_date,
)

Many things happen during initialisation

The inputs get assigned as attributes, and `birth_date` is converted to datetime.

In [ ]:
patient.patient_id, patient.gender, patient.birth_date

### Datetime timezones

Throughout the codebase, datetimes are timezone aware and a UTC timezone is chosen by default. This is so that datetimes are FHIR compliant https://www.hl7.org/fhir/datatypes.html#dateTime

Time zones are added automatically. If a different time zone is required, some work will need to be done to expose the time zone as a parameter and allow the user to change it. To see where timezones are being set, search for `tz` in the `patient_abm` codebase.

### The base `Agent` class

The `PatientAgent` class is a subclass of `Agent`.

In [ ]:
patient.__class__.__bases__[0]

This means that it has an internal unique `id` and `created_at` timestamp which gets automatically set, and you can see the values in the string representation of `patient`

In [ ]:
patient

In [ ]:
patient.id, patient.created_at

In future, a more patient-specific `PatientAgent.__repr__` could be implemented.

### The patient `record` and `PatientRecordEntry`

A `patient` has an attribute called `record` which represents its health record. The `record` is a python list. Each item in the list is called an "entry", and it is an abstraction of a FHIR resource. The `record` gets appended to over the course of a simulation.

An entry is an instance of a `PatientRecordEntry` dataclass. This dataclass has a number of attributes:

  - `real_time`: the real datetime when the entry was generated
  - `patient_time`: the datetime from the patient's perspective when the entry was generated
  - `environment_id`: if an entry was added during the simulation, then this is the ID of the environment that that patient was interacting with when the entry was generated
  - `interactions`: the names of interaction functions that were applied when the patient was interacting with the environment (the intelligence layer in a single simulation step could apply multiple interactions)
  - `simulation_step`: the step of the simulation generating this entry. In a single step, multiple entries might be generated, so this groups them together
  - `fhir_resource_time`: ***!!!TO BE REMOVED!!!***
  - `fhir_resource_type`: the FHIR resource type of the entry
  - `fhir_resource`: raw FHIR resource data
  - `record_index`: the index in the record list
  - `entry_id`: an ID for the entry
  - `tag`: a name for the entry - can be used as a human readable way to search the record for multiple related entries
  - `entry`: the actual content of the entry, a dictionary which is an abstraction of a FHIR resource

This `patient` record has a single entry which is just the patient's demographics

In [ ]:
patient.record

It is a little easier to see using the dataclasses function asdict

In [ ]:
asdict(patient.record[0])

### The patient `profile`

The first record entry is special and it always the patient demographics. In fact the first entry is assigned to the patient profile attribute

In [ ]:
patient.profile == patient.record[0]

In [ ]:
asdict(patient.profile)

Note that `'fhir_resource_type': 'Patient'`, i.e. this is supposed to correspond to a FHIR Patient resource.

### Modifying the patient profile and the `kwargs` argument

How can more data be added to the patient profile? Suppose we wanted to add `age` as an attribue to the patient. Currently it is not

In [ ]:
patient.age

The `PatientAgent` init method accepts keyword arguments, which get set as attributes.

In [ ]:
patient = PatientAgent(
    patient_id=patient_id,
    gender=gender,
    birth_date=birth_date,
    age=65,
)

In [ ]:
patient.age

This enables flexible setting of new patient attributes, and it can even be done on the fly from the configuration script `config.json`.

Does `age` get added to the patient profile, or equivalently, its record?

In [ ]:
# asdict(patient.profile)
# we expect to see it in patient.profile.entry
patient.profile.entry

There is no age.

In order to instruct the init method to add an attribute to the patient profile, and therefore the record, the prefix `patient__` must be used.

In [ ]:
patient = PatientAgent(
    patient_id=patient_id,
    gender=gender,
    birth_date=birth_date,
    patient__age=65,
)

In [ ]:
patient.profile.entry

Now it is there without the prefix, and in the record

In [ ]:
patient.record[0].entry

For bookkeeping, it is also added as an attribute (but with the prefix)

In [ ]:
patient.patient__age

In [ ]:
patient.age

### Converting the patient `record` to a FHIR resource

Since the patient record is the object that gets converted to a FHIR resource later, adding fields in like this could be useful if we wanted to add extensions to the FHIR Patient resource, such as ethinicity. Note that, while currently it is possible to add these additional fields on the fly, they won't actually be included in the FHIR converted resource, this is because we are only selecting specific fields to convert to FHIR so that passes validation. We leave more flexible conversions for a future version (it would involve making changes to the `convert_patient_record_entry_to_fhir` function in the `patient_abm.data_handler.fhir` module)

To see how this looks when converted to FHIR:

In [ ]:
fhir_resource = convert_patient_record_entry_to_fhir(patient.record[0], patient)

In [ ]:
# original
patient.record[0].entry

In [ ]:
# converted
fhir_resource

Notice that age has been removed, as stated above.

There is still a way to include additional data to the FHIR output, and that is the role of the `fhir_resource` field in `PatientRecordEntry`. For instance, suppose we wanted to add an contact information to the patient profile resource:

In [ ]:
# snippet copied from https://www.hl7.org/fhir/patient-example.json.html
contact = {
    "telecom": [
        {
            "system": "phone",
            "value": "(03) 5555 6473",
            "use": "work",
            "rank": 1
        }
    ]
}

Currently the only way to add this to the `fhir_resource` field of the patient profile is to add it after initialising the patient:

In [ ]:
patient.record[0].fhir_resource = contact

In [ ]:
patient.record[0]

Now when we call `convert_patient_record_entry_to_fhir`, it will first generate the FHIR resource from `a = patient.record[0].entry`, and then combine that with `b = patient.record[0].fhir_resource`, i.e. the output is `{**a, **b}`

In [ ]:
convert_patient_record_entry_to_fhir(patient.record[0], patient)

In this case the output was as expected because the joining `{**a, **b}` was simple. In general though, care needs to be taken because the operation might lead to unintended results. We are highlighting this now but leave more complex conversions for future work.

### The `start_time`, `name`, `alive`, and `inpatient` attributes

These attributes are automatically set.

In [ ]:
print(patient.start_time) # set to created_at by default
print(patient.name) # set to patient_id by default
print(patient.alive) # set to True by default
print(patient.inpatient) # set to None by default

Alternatively, they can be set to custom values when initialising the patient agent

In [ ]:
patient = PatientAgent(
    patient_id=patient_id,
    gender=gender,
    birth_date=birth_date,
    start_time="2020/05/01",
    name="Sarah",
)

In [ ]:
patient.start_time, patient.name

`start_time` is the time in patient-perspective when their pathway starts. It is sets the `patient_time` value in the first record entry (i.e. patient profile) (above this was the same as created_at)

In [ ]:
patient.profile.patient_time

The patient name does not get added to the profile

In [ ]:
patient.profile.entry

This could be added in future, and then the name could also appear in the converted FHIR resource. Though note that names in FHIR Patient resource are very structured
```
"name" = [
    {
        "use": "official",
        "family": "Chalmers",
        "given": [
            "Peter",
            "James"
        ]
    }
]
```
so this mapping would need to be added

The `alive` attribute is a boolean which gets starts as True by default, but gets set to False if the patient `death` interaction is applied (see `patient_abm.intelligence.interactions.default.death`, and note that this function also adds the death date to the patient profile, which gets converted to `deceasedDate` in the FHIR Patient resource). This bool is checked at every step of the simulation (see `patient_abm.simulation.run.run_patient_simulation`) and terminates the simulation if is it False. 

If the patient is an inpatient it could greatly affect the dynamics of the simulation and the pathway. The `inpatient` attribute is currently a placeholder, and could be used in the simulation to hold information about whether the patient is an inpatient. E.g. if patient is not an inpatient, then `patient.inpatient = None`, otherwise:

`patient.inpatient = {"time_admitted": "xxx", "location": "xxx"}`

We leave the specifics to be developed further in the future.

### The `conditions`, `medications`, and `actions` attributes

The conditions attributes table tracks patient conditions such as diseases.

The medications table tracks patient medications.

The actions table tracks outsanding patient actions such as scheduled appointments.

These attributes are represented as pandas dataframes, and will currently be empty

In [ ]:
patient.conditions

In [ ]:
patient.medications

In [ ]:
patient.actions

### Adding new attributes to the tables, e.g. adding severity to the conditions table

The tables come with predefined columns, however it is easy to add custom columns. For example, suppowe wanted to add "severity" to the conditions table. There is an argument called
`conditions_custom_fields` where we can list new column names:

In [ ]:
patient = PatientAgent(
    patient_id=patient_id,
    gender=gender,
    birth_date=birth_date,
    conditions_custom_fields=["severity"],
)

In [ ]:
# severity is added
patient.conditions

Similarly there are `medications_custom_fields` and `actions_custom_fields`

The columns that appear by default are:

In [ ]:
for attr_name in ["conditions", "medications", "actions"]:
    print(f"Default {attr_name} columns:")
    print(
        PatientAgent._core_dataframe_fields 
        + PatientAgent._dataframe_attributes_to_kwargs[attr_name]["additional_core_fields"]
    )
    print()

These tables can be populated in two ways:

(1) A non-empty table can be supplied during initialisation

(2) When a patient's record gets updated / appended, these tables also get updated

Let's talk about (1) first, in the context of initialising a patient with comorbities.

### Initialising a patient with comorbidities

Suppose the patient has two active comorbidities: diabetes and underactive thyroid. For each we need to supply a name and the start date of the disease

In [ ]:
diabetes = {
    "name": "diabetes",
    "start": "2015-09-24",
}
underactive_thyroid = {
    "name": "underactive_thyroid",
    "start": "2019-07-18",
}

In [ ]:
comorbidities = [diabetes, underactive_thyroid]

`comorbidities` can be supplied as a list of dictionaries to the patient agent constructor as the conditions argument.

In [ ]:
patient = PatientAgent(
    patient_id=patient_id,
    gender=gender,
    birth_date=birth_date,
    conditions=comorbidities,
)

The conditions get loaded into the table and the extra columns are added. Notice that `active` is set to True. This is because the diseases have no end date 

In [ ]:
patient.conditions

We could also add extra severity column manually

In [ ]:
patient = PatientAgent(
    patient_id=patient_id,
    gender=gender,
    birth_date=birth_date,
    conditions=comorbidities,
    conditions_custom_fields=["severity"],
)

In [ ]:
patient.conditions

Or implicitly by including it in a comorbidity

In [ ]:
diabetes["severity"] = "high"

In [ ]:
comorbidities

In [ ]:
patient = PatientAgent(
    patient_id=patient_id,
    gender=gender,
    birth_date=birth_date,
    conditions=comorbidities,
)

In [ ]:
patient.conditions

A condition becomes inactive if the "end" date is a date (i.e. a timestamp for when the patient recovers from the disease, assumed to be in the "past" of the patient's "current time"). It's best to see this in action by updating the patient record.

Let's first have a look at the current record

In [ ]:
patient.record

Note that currently these conditions are not added as record entries (this could be changed in the future if required).

Instead let's add a record entry that signifies a recovery from diabetes

In [ ]:
diabetes["end"] = "2021-03-24"
diabetes

In summary, a patient can be initialised with comorbidities by creating a list or dataframe of diseases, with at least name and start columns, and then passing this to the PatientAgent init method.

### Updating the patient record

We can update the patient record with this entry. Each new entry must have at least `name`, `start` fields as well as `resource_type`. The disease entries are resource type Condition. Adding that:

In [ ]:
diabetes["resource_type"] = "Condition"

In [ ]:
patient.update([diabetes], wrapped=False)

Look at the record, the new data has been added.

In [ ]:
patient.record

Look at the conditions table, diabetes is no longer active

In [ ]:
patient.conditions

The table has preserves unique (name, start) pair tuples. So we could try and add `underactive_thyroid` to the record. Since it's not in the record, it will get added there, but not change the conditions table. Again we first have to add in resource_type

In [ ]:
underactive_thyroid["resource_type"] = "Condition"

In [ ]:
patient.update([underactive_thyroid], wrapped=False)

In [ ]:
patient.record

In [ ]:
patient.conditions

Note a patient conditons can also be initialised by supplying a dataframe

In [ ]:
df = patient.conditions.copy(deep=True)

In [ ]:
patient = PatientAgent(
    patient_id=patient_id,
    gender=gender,
    birth_date=birth_date,
    conditions=df,
)

In [ ]:
patient.conditions

The actions and medications tables work similarly. Actions is triggered by FHIR resource types Appointment and ServiceRequest, whereas medications is triggered by MedicationRequest.

The intelligence layer can act on active elements and the mark them as inactive by adding an end date to an entry. Currently a new entry has to be appended to the record to trigger changes in these tables. If the intelligence layer adds an "end" date to an exisitng record (without appending) then no change will be made to the table. This could be changed in a future version.

### Validating the patient record

Every time `patient.update` is called, a list of new entries is added one by one to the exisiting record and two checks are made:
- a check of whether the entry already exists in the record
- a check of whether a relevant date is in the right order

For the first check, there is an option to "skip" or "add" the duplicate enrty, and it controlled by the `skip_existing` flag.

We know from above that `patient.profile.entry` is already in the record, we could try readding that, but it would fail because it is doesn't have `name` and `start` fields.

Instead let's try to add diabetes twice

In [ ]:
patient.update([diabetes], wrapped=False, skip_existing=False)

In [ ]:
patient.record

In [ ]:
patient.update([diabetes], wrapped=False, skip_existing=False)

In [ ]:
patient.record

Now if we skip existing

In [ ]:
patient.update([diabetes], wrapped=False, skip_existing=True)

In [ ]:
patient.record

The whole time, the conditions table stayed as is

In [ ]:
patient.conditions

### Wrapping the entry

Notice we've been using `wrapped=False` in the update method above. That's because, before adding to the patient record, an entry must be wrapped using the function `wrap_fhir_resource`. When `wrapped=False` it gets wrapped as

In [ ]:
entry = diabetes

In [ ]:
wrapped_entry = wrap_fhir_resource(
    entry,
    patient_time=entry["start"] if entry.get("end") is None else entry["end"],                   
)

In [ ]:
wrapped_entry

This then gets passes to the `PatientRecordEntry` class constructor.

### Simulation entries from the intelligence layer interactions

In the simulation, it is expected that the interaction layer will generate the inner `entry` dictionaries, which look like `diabetes` or `underactive_thyroid`. The simulation then automatically wraps the entry before updating the patient record. Have a look at
`patient_abm.simulation.run.update_patient_and_environment` to see how this wrapping is done.

### The internal record representation

To save the user from writing raw FHIR, we have created an internal representation of FHIR resources, namely, the patient record (unwrapped) entries. These are dictionaries which must have three keys (as discussed above):
- `name`
- `start`
- `resource_type`

Other keys that can be used are the ones in the tables and they can be resource type dependent, e.g.
- `end`
- `code`

etc.

They get used when converting the patient record to FHIR, the conversion process can be seen in
`patient_abm.data_handler.fhir.convert_patient_record_entry_to_fhir`. Currently conversion to / from the following FHIR resources is supported:
- Patient
- Encounter
- Condition
- Observation
- Procedure
- MedicationRequest
- ServiceRequest
- Appointment

Within those, only certain fields are populated. What we have built so far is a very simplified version of this representation, must more complexity could be added in order to produce richer FHIR outputs, for example, more fields, data from environments, and linking between resources (e.g. link to encounter resource from other related resources). Further, the FHIR MedicationRequest doesn't have an obvious end date field, instead it could be calculaeted from the start date plus expected duration of the medication, given the `dispenseRequest.expectedSupplyDuration`.

### Logging the patient state

During the simulation, the patient state is written to the logger. The informaiton that's written is the output of the `log_state` method

In [ ]:
patient.log_state(print_only=True)

The n last record entries can also be logged

In [ ]:
patient.log_state(print_only=True, log_last_n_record_entries=1)

### Converting patient record to FHIR bundle

Can validate using offline method (the python `fhir.resources` library)

In [ ]:
bundle = generate_patient_fhir_bundle(
    patient,
    validate=True,
    server_url=None,
)

In [ ]:
bundle

Or online by sending the data to the HAPI FHIR server

In [ ]:
HAPI_FHIR_SERVER_4

In [ ]:
bundle = generate_patient_fhir_bundle(
    patient,
    validate=True,
    server_url=HAPI_FHIR_SERVER_4,
)

The online method is far more stringent. There are cases where the offline library passes but the online version fails. A more robust mapping is left for future work.

In [ ]:
bundle

### Initialising a patient from FHIR data

We can initialize a patient from FHIR data. Suppose we wish to initialise the patient from the FHIR bundle we just created, we can do that by simply calling the `from_fhir` method

In [ ]:
from patient_abm.data_handler.fhir import convert_fhir_to_patient_record_entry

In [ ]:
patient_from_fhir = PatientAgent.from_fhir(
    bundle,
    resource_type="Bundle",
    patient_id=None,
    start_time=None,
)

In [ ]:
patient_from_fhir.record

In [ ]:
patient_from_fhir.conditions

In this way the PatientAgent could be initialised from collections of FHIR resources, such as PRSB core information standards profiles, or Care Connect profiles.

# Save and load patient agent class

Save and load patient agent class as tar file. Only `serialisable_attributes` are saved

In [ ]:
# For now these are
patient.serialisable_attributes

In [ ]:
patient_file = PATIENT_ABM_DIR / "notebooks" / "patient_agent.tar"
patient_file

In [ ]:
patient.save(patient_file)

In [ ]:
patient_ = PatientAgent.load(patient_file)

In [ ]:
dfs = ["conditions", "medications", "actions"]

for attr_name in PatientAgent.serialisable_attributes:
    if attr_name in dfs:
        assert (
            getattr(patient, attr_name)
            .fillna("n/a")
            .equals(getattr(patient_, attr_name).fillna("n/a"))
        )
    else:
        assert getattr(patient, attr_name) == getattr(patient_, attr_name)